In [1]:
import numpy as np

from functools import reduce
from operator import xor
from itertools import chain


In [2]:
# two set of intervals intersection 
def intersections(a,b):
    if b==[[]]:
        return a
    ranges = []
    i = j = 0
    while i < len(a) and j < len(b):
        a_left, a_right = a[i]
        b_left, b_right = b[j]

        if a_right < b_right:
            i += 1
        else:
            j += 1

        if a_right >= b_left and b_right >= a_left:
            end_pts = sorted([a_left, a_right, b_left, b_right])
            middle = [end_pts[1], end_pts[2]]
            ranges.append(middle)

    ri = 0
    while ri < len(ranges)-1:
        if ranges[ri][1] == ranges[ri+1][0]:
            ranges[ri:ri+2] = [[ranges[ri][0], ranges[ri+1][1]]]
        ri += 1
    return ranges
    



def exclude_gaps(set1, set2): #set2 is set of gaps [[],[],[]]

    l = sorted((reduce(xor, map(set, chain(set1 , set2)))))
    XOR=[l[i:i + 2] for i in range(0, len(l), 2)]

    return intersections(XOR, set1)


def merge_overlap(arr):
    # Sort intervals based on start values
    arr.sort(key=lambda x: x[0])

    res = [arr[0]]

    for i in range(1, len(arr)):
        last = res[-1]
        curr = arr[i]

        # If current overlaps with the last
        # merged, merge them
        if curr[0] <= last[1]:
            last[1] = max(last[1], curr[1])
        else:
            # Add current to the result
            res.append(curr)

    return res

In [3]:
with open('grch38.gaps.bed', 'r') as f:
    l=f.readlines()

l=[l[i].replace('\n','').split('\t') for i in range(len(l))]

In [4]:
dct={}
chr_mas=['chr'+str(i) for i in range(1,23)]

for chr in chr_mas:
    dct[chr]=[]

for j in l:
    if j[0] in chr_mas:
        dct[j[0]].append([int(j[1]), int(j[2])])

In [5]:
dct_bed={}
for chr in chr_mas:

    m=merge_overlap(dct[chr])

    
    m1=min([m[i][1] for i in range(len(m))])
    m2=max([m[i][0] for i in range(len(m))])
    

    dct_bed[chr]=exclude_gaps([[m1+1,m2-1]], dct[chr])

In [6]:
for chr in chr_mas:
    with open(chr+'.grch38.bed', 'w') as f:
        for j in dct_bed[chr]:
            if j[1]-j[0]>50000:
                f.write(chr+'\t'+str(j[0])+'\t'+str(j[1])+'\n')